In [7]:
import requests
import json
import re
from bs4 import BeautifulSoup
import wikipedia

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

URL = "https://ru.wikipedia.org/wiki/Список_русских_городов_дальних_и_ближних"
OUTPUT_FILE = "russian_historical_cities.json"

wikipedia.set_lang("ru")


# ---------------------------
# Утилиты
# ---------------------------

def clean_text(text: str) -> str:
    return re.sub(r"\s+", " ", text.strip())


def split_modern_names(text: str) -> list[str]:
    """
    Убирает [число], делит по / и ,
    """
    text = re.sub(r"\[\d+\]", "", text)
    parts = re.split(r"[\/,]", text)
    return [p.strip() for p in parts if p.strip()]


def extract_cite_number(text: str) -> str | None:
    m = re.search(r"\[(\d+)\]", text)
    return m.group(1) if m else None


# ---------------------------
# Работа со сносками
# ---------------------------

def parse_citations(soup: BeautifulSoup) -> dict:
    """
    Возвращает словарь:
    { "19": "Повесть временных лет, Лаврентьевская летопись" }
    """
    citations = {}
    for li in soup.select("li[id^=cite_note-]"):
        key = li["id"].replace("cite_note-", "")
        text = clean_text(li.get_text(" "))
        citations[key] = text
    return citations


def chronicle_from_citation(citation_text: str) -> str | None:
    """
    Очень аккуратная эвристика:
    мы НЕ выдумываем, а просто сохраняем формулировку источника
    """
    if not citation_text:
        return None

    # часто встречающиеся летописи
    keywords = [
        "летопись",
        "Повесть временных лет",
        "Ипатьевская",
        "Лаврентьевская",
        "Новгородская"
    ]

    for k in keywords:
        if k.lower() in citation_text.lower():
            return citation_text

    return citation_text


def extract_foundation_info(page) -> dict | None:
    """
    Пытается извлечь информацию об основании города из инфобокса.
    Возвращает:
      { "year": int | None, "text": str }
    или None
    """
    try:
        html = page.html()
        soup = BeautifulSoup(html, "lxml")

        infobox = soup.find("table", class_="infobox")
        if not infobox:
            return None

        labels = [
            "Основан",
            "Основана",
            "Дата основания",
            "Первое упоминание"
        ]

        for row in infobox.find_all("tr"):
            th = row.find("th")
            td = row.find("td")
            if not th or not td:
                continue

            label = th.get_text(strip=True)
            if any(l.lower() in label.lower() for l in labels):
                text = clean_text(td.get_text(" "))

                # ищем год
                m = re.search(r"(\d{3,4})", text)
                year = int(m.group(1)) if m else None

                return {
                    "year": year,
                    "text": text
                }

    except Exception:
        return None

    return None

# ---------------------------
# Wikipedia API
# ---------------------------
import warnings
from wikipedia.exceptions import DisambiguationError, PageError

# подавляем warning от wikipedia
warnings.filterwarnings(
    "ignore",
    message="No parser was explicitly specified"
)


def get_wiki_city_data(city_name: str, verbose: bool = True) -> dict:
    if verbose:
        print(f"    🔎 Поиск города: {city_name}")

    def empty_result(reason: str):
        if verbose:
            print(f"      ⛔ Пропуск города: {reason}")
        return {
            "name": city_name,
            "wiki_url": None,
            "coords": {"lat": None, "lon": None},
            "image_640": None
        }

    # --- попытка №1 ---
    try:
        page = wikipedia.page(city_name, auto_suggest=False)
        if verbose:
            print(f"      ✔ Найдена страница: {page.url}")
    except DisambiguationError as e:
        if verbose:
            print(f"      ⚠ Disambiguation ({len(e.options)} вариантов)")

        # пытаемся выбрать ПЕРВЫЙ ОСМЫСЛЕННЫЙ вариант (с пробелом или уточнением)
        candidates = [opt for opt in e.options if " " in opt or "," in opt]

        if not candidates:
            return empty_result("неоднозначное название (без надёжного варианта)")

        candidate = candidates[0]
        if verbose:
            print(f"      ➜ Пробуем уточнение: {candidate}")

        try:
            page = wikipedia.page(candidate, auto_suggest=False)
            if verbose:
                print(f"      ✔ Уточнённая страница: {page.url}")
        except DisambiguationError:
            return empty_result("повторная неоднозначность")
        except PageError:
            return empty_result("страница не найдена")
        except Exception as e:
            return empty_result(f"ошибка Wikipedia API: {e}")

    except PageError:
        return empty_result("страница не найдена")
    except Exception as e:
        return empty_result(f"ошибка Wikipedia API: {e}")

    # ---------- координаты ----------
    lat, lon = None, None
    try:
        coords = page.coordinates
        if coords:
            lat, lon = float(coords[0]), float(coords[1])
            if verbose:
                print(f"      📍 Координаты: {lat}, {lon}")
    except Exception:
        if verbose:
            print(f"      ℹ Координаты отсутствуют")

    # ---------- изображение ----------
    image_url = None
    try:
        for img in page.images:
            if img.lower().endswith((".jpg", ".jpeg", ".png")):
                image_url = img
                if verbose:
                    print(f"      🖼 Найдено изображение")
                break
        if not image_url and verbose:
            print(f"      ℹ Изображение не найдено")
    except Exception:
        if verbose:
            print(f"      ℹ Ошибка при получении изображения")

    foundation = extract_foundation_info(page)

    if verbose and foundation:
        print(f"      📜 Основание: {foundation['text']}")
    elif verbose:
        print(f"      ℹ Дата основания не указана")

    return {
        "name": city_name,
        "wiki_url": page.url,
        "coords": {"lat": lat, "lon": lon},
        "image_640": image_url,
        "foundation": foundation
    }


# ---------------------------
# Основной парсер
# ---------------------------

def main():
    response = requests.get(URL, headers=HEADERS, timeout=30)
    
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    citations = parse_citations(soup)

    table = soup.find("table")
    if not table:
        raise RuntimeError("Таблица не найдена")

    result = []

    for row in table.find_all("tr"):
        cols = row.find_all("td")
        if len(cols) < 6:
            continue

        origin_type = clean_text(cols[0].get_text())
        historical_raw = [clean_text(cols[i].get_text()) for i in range(1, 5)]
        historical_raw = [h for h in historical_raw if h]

        modern_raw = clean_text(cols[5].get_text())
        cite_number = extract_cite_number(modern_raw)

        chronicle = None
        if cite_number and cite_number in citations:
            chronicle = chronicle_from_citation(citations[cite_number])

        historical_names = [
            {
                "name": h,
                "chronicle": chronicle
            }
            for h in historical_raw
        ]

        modern_cities = []
        for modern_name in split_modern_names(modern_raw):
            modern_cities.append(get_wiki_city_data(modern_name))

        result.append({
            "origin_type": origin_type,
            "historical_names": historical_names,
            "modern_cities": modern_cities
        })

    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

    print(f"Готово: {OUTPUT_FILE}")


if __name__ == "__main__":
    main()

    🔎 Поиск города: Видин
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%92%D0%B8%D0%B4%D0%B8%D0%BD
      📍 Координаты: 43.98869, 22.874058
      🖼 Найдено изображение
      ℹ Дата основания не указана
    🔎 Поиск города: Видин или Калафат
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Велико-Тырново
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%92%D0%B5%D0%BB%D0%B8%D0%BA%D0%BE-%D0%A2%D1%8B%D1%80%D0%BD%D0%BE%D0%B2%D0%BE
      📍 Координаты: 43.08333, 25.65
      🖼 Найдено изображение
      ℹ Дата основания не указана
    🔎 Поиск города: Силистра
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%A1%D0%B8%D0%BB%D0%B8%D1%81%D1%82%D1%80%D0%B0
      📍 Координаты: 44.11667, 27.26667
      🖼 Найдено изображение
      ℹ Дата основания не указана
    🔎 Поиск города: Мэчин или Вичина (Исакча)
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Килия-Веке
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%9A%D0%B8%D0%BB%D0%

      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%94%D0%B2%D0%B5%D1%80%D0%B5%D0%BD
      ℹ Координаты отсутствуют
      ℹ Изображение не найдено
      ℹ Дата основания не указана
    🔎 Поиск города: Корсунь-Шевченковский
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D1%80%D1%81%D1%83%D0%BD%D1%8C-%D0%A8%D0%B5%D0%B2%D1%87%D0%B5%D0%BD%D0%BA%D0%BE%D0%B2%D1%81%D0%BA%D0%B8%D0%B9
      📍 Координаты: 49.42611, 31.28056
      🖼 Найдено изображение
      📜 Основание: 1032
    🔎 Поиск города: Триполье
      ⚠ Disambiguation (3 вариантов)
      ➜ Пробуем уточнение: Культура Триполье-Кукутень
      ✔ Уточнённая страница: https://ru.wikipedia.org/wiki/%D0%9A%D1%83%D0%BB%D1%8C%D1%82%D1%83%D1%80%D0%B0_%D0%A2%D1%80%D0%B8%D0%BF%D0%BE%D0%BB%D1%8C%D0%B5_%E2%80%94_%D0%9A%D1%83%D0%BA%D1%83%D1%82%D0%B5%D0%BD%D1%8C
      ℹ Координаты отсутствуют
      🖼 Найдено изображение
      ℹ Дата основания не указана
    🔎 Поиск города: Канев
      ✔ Найдена страница: https://ru.wikipedi

      ℹ Координаты отсутствуют
      🖼 Найдено изображение
      ℹ Дата основания не указана
    🔎 Поиск города: близ устья Ромна или Клепачи
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Ромны
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%A0%D0%BE%D0%BC%D0%BD%D1%8B
      📍 Координаты: 50.74281279, 33.48790812
      🖼 Найдено изображение
      📜 Основание: XI век
    🔎 Поиск города: Новые Санжары
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%9D%D0%BE%D0%B2%D1%8B%D0%B5_%D0%A1%D0%B0%D0%BD%D0%B6%D0%B0%D1%80%D1%8B
      📍 Координаты: 49.33539098, 34.31619855
      🖼 Найдено изображение
      📜 Основание: XVII век
    🔎 Поиск города: близ устья Орели
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Хотень или Гадяч
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Ницаха
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%9D%D0%B8%D1%86%D0%B0%D1%85%D0%B0
      📍 Координаты: 50.4597432, 35.18219479
      🖼 Най

      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%98%D0%B2%D0%B0%D0%BD%D1%8C%D0%B5
      📍 Координаты: 50.46778, 25.74889
      🖼 Найдено изображение
      📜 Основание: 1412
    🔎 Поиск города: Иваниничи или Иван
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Кременец
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%9A%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%B5%D1%86
      📍 Координаты: 50.0974514, 25.73509619
      🖼 Найдено изображение
      📜 Основание: 1226
    🔎 Поиск города: Белз
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%91%D0%B5%D0%BB%D0%B7
      📍 Координаты: 50.38333, 24.01667
      🖼 Найдено изображение
      📜 Основание: 1030
    🔎 Поиск города: Дубище на Случи
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Дубецко[пол.] или Дубище на Стыре
      ⛔ Пропуск города: ошибка Wikipedia API: 'fullurl'
    🔎 Поиск города: Теребовля
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D1%80%D0%B5%D0%B1

      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%9C%D1%8F%D0%B4%D0%B8%D0%BD%D0%B8%D0%BD%D0%BA%D0%B0%D0%B9
      📍 Координаты: 54.53888889, 25.65
      🖼 Найдено изображение
      📜 Основание: 1387
    🔎 Поиск города: Крево
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%9A%D1%80%D0%B5%D0%B2%D0%BE
      📍 Координаты: 54.310375, 26.2865472
      🖼 Найдено изображение
      📜 Основание: XIII век
    🔎 Поиск города: Лоск
      ⚠ Disambiguation (6 вариантов)
      ➜ Пробуем уточнение: Лоск (рынок)
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Гольшаны
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D0%BB%D1%8C%D1%88%D0%B0%D0%BD%D1%8B
      📍 Координаты: 54.255766, 26.014276
      🖼 Найдено изображение
      📜 Основание: 1280
    🔎 Поиск города: Березуеск
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%91%D0%B5%D1%80%D0%B5%D0%B7%D1%83%D0%B9%D1%81%D0%BA
      📍 Координаты: 56.03556, 34.48861
      🖼 Найдено изображение


      📍 Координаты: 54.48333, 34.98333
      🖼 Найдено изображение
      📜 Основание: в 1231 году
    🔎 Поиск города: Никольское или Яблоновец
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Оболенск
      ⚠ Disambiguation (5 вариантов)
      ⛔ Пропуск города: неоднозначное название (без надёжного варианта)
    🔎 Поиск города: Сутоки или Сутейск
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Ясенки (сельское поселение «Деревня Редькино»)
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Острино или Острий
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Микулино или Микулинцы
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Стародуб
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%B0%D1%80%D0%BE%D0%B4%D1%83%D0%B1
      📍 Координаты: 52.58333, 32.76667
      🖼 Найдено изображение
      📜 Основание: 1080 [ источник не указан 2667 дней ]
    🔎 Поиск города: Мглин
      ✔ Найдена страница: h

      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Козельск или Ивановское
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Вятическ
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%92%D1%8F%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA
      ℹ Координаты отсутствуют
      ℹ Изображение не найдено
      ℹ Дата основания не указана
    🔎 Поиск города: Вержавск
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%92%D0%B5%D1%80%D0%B6%D0%B0%D0%B2%D1%81%D0%BA
      📍 Координаты: 55.366139, 31.94556
      🖼 Найдено изображение
      📜 Основание: 1136
    🔎 Поиск города: Старая Рязань
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%B0%D1%80%D0%B0%D1%8F_%D0%A0%D1%8F%D0%B7%D0%B0%D0%BD%D1%8C
      📍 Координаты: 54.39694, 40.4275
      🖼 Найдено изображение
      📜 Основание: XI век
    🔎 Поиск города: Новый Ольгов
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%9D%D0%BE%D0%B2%D1%8B%D0%B9_%D0%9E%D0%BB%D1%8C%D0%B3%D0%BE%D0%B2

      📍 Координаты: 56.23528, 42.24417
      🖼 Найдено изображение
      📜 Основание: 1135 — 1138
    🔎 Поиск города: Гороховец
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D1%85%D0%BE%D0%B2%D0%B5%D1%86
      📍 Координаты: 56.20278, 42.6925
      🖼 Найдено изображение
      📜 Основание: 1168
    🔎 Поиск города: Бережец
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%91%D0%B5%D1%80%D0%B5%D0%B6%D0%B5%D1%86
      📍 Координаты: 56.1501723, 42.9376227
      ℹ Изображение не найдено
      📜 Основание: начало XII века
    🔎 Поиск города: Нижний Новгород
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%9D%D0%B8%D0%B6%D0%BD%D0%B8%D0%B9_%D0%9D%D0%BE%D0%B2%D0%B3%D0%BE%D1%80%D0%BE%D0%B4
      📍 Координаты: 56.32694, 44.0075
      🖼 Найдено изображение
      📜 Основание: 1221
    🔎 Поиск города: Курмыш
      ⚠ Disambiguation (14 вариантов)
      ➜ Пробуем уточнение: Курмыш (приток Вашки)
      ✔ Уточнённая страница: https://ru.wikipedia.org

      🖼 Найдено изображение
      📜 Основание: 1374 год
    🔎 Поиск города: Кислино
      ⚠ Disambiguation (4 вариантов)
      ⛔ Пропуск города: неоднозначное название (без надёжного варианта)
    🔎 Поиск города: Малоярославец или другой пункт на реке Луже
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Боровск
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%91%D0%BE%D1%80%D0%BE%D0%B2%D1%81%D0%BA
      📍 Координаты: 55.2, 36.48333
      🖼 Найдено изображение
      📜 Основание: в 1358 году
    🔎 Поиск города: Никольское
      ⚠ Disambiguation (315 вариантов)
      ➜ Пробуем уточнение: Никольское 1-е
      ⛔ Пропуск города: повторная неоднозначность
    🔎 Поиск города: Оболенск или Городище
      ⛔ Пропуск города: страница не найдена
    🔎 Поиск города: Одоев
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%9E%D0%B4%D0%BE%D0%B5%D0%B2
      📍 Координаты: 53.94222, 36.69972
      🖼 Найдено изображение
      📜 Основание: 1375
    🔎 Поиск города: Белёв


      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%92%D0%B5%D0%BB%D0%B8%D0%BA%D0%B8%D0%B5_%D0%9B%D1%83%D0%BA%D0%B8
      📍 Координаты: 56.35, 30.51667
      🖼 Найдено изображение
      📜 Основание: 1166
    🔎 Поиск города: Старая Русса
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%B0%D1%80%D0%B0%D1%8F_%D0%A0%D1%83%D1%81%D1%81%D0%B0
      📍 Координаты: 57.98333, 31.35
      🖼 Найдено изображение
      📜 Основание: 1167
    🔎 Поиск города: Курско
      ⚠ Disambiguation (6 вариантов)
      ➜ Пробуем уточнение: Курск (значения)
      ⛔ Пропуск города: повторная неоднозначность
    🔎 Поиск города: Копорье
      ⚠ Disambiguation (8 вариантов)
      ⛔ Пропуск города: неоднозначное название (без надёжного варианта)
    🔎 Поиск города: Ям
      ✔ Найдена страница: https://ru.wikipedia.org/wiki/%D0%AF%D0%BC
      ℹ Координаты отсутствуют
      🖼 Найдено изображение
      ℹ Дата основания не указана
    🔎 Поиск города: Порхов
      ✔ Найдена страница: https://ru